# Tarea 1

1.- Estudiar la informacion contenida en el archivo /.ipython/profile_default/security/ipcontroller-client.json y explicar en un parrafo que significa.

El archivo ipcontroller-client.json es la información de conexión del cliente cluster   

{  
    "url":"tcp://192.168.1.123:56951",  
    "exec_key":"26f4c040-587d-4a4e-b58b-030b96399584",  
    "ssh":"user@example.com",  
    "location":"192.168.1.123"  
}  

Contiene la url y el puerto de conexión, el ssh que sirve para indicar el lugar donde se encuentran los motores fuera LAN donde se encuentra el controlador.

2.- Investigar el uso de los metodos apply,apply_sync de las instancias DirectView (el objeto dview) y discutir las diferencias con map y map_sync, por medio de ejemplos.

> apply es el método principal para hacer execuciones remotas, cuando el objeto DirectView.block = True envia el comando al controlador, el cual coloca el comando en la cola de motores para su ejecución.

In [3]:
import numpy as np
import ipyparallel as ipp

In [4]:
c = ipp.Client()
c.ids

[0, 1, 2, 3]

In [5]:
dview = c[:] #se asignan todos los motores
dview.block=False
dview['a'] = 5
dview['b'] = 10
dview.apply_sync(lambda x: a+b+x, 27)

[42, 42, 42, 42]

> Si el objeto DirectView.block = False, envía el comando para ser ejecutado y regresa un resultado asíncrono inmediatamente, es decir, no espera a que termine la ejecución en otro motor.

In [6]:
dview.block=False
dview['a'] = 5
dview['b'] = 10
dview.apply(lambda x: a+b+x, 27)

<AsyncResult: <lambda>>

In [7]:
dview.block=False
dview['a'] = 5
dview['b'] = 10
dview.apply_sync(lambda x: a+b+x, 27)

[42, 42, 42, 42]

> La función map() permite aplicar una función a una secuencia elemento-a-elemento (de forma serial)

In [8]:
def cube(x):
    return x**3

In [9]:
serial_result = map(cube, range(1,7))
print serial_result

[1, 8, 27, 64, 125, 216]


> La función map_sync() permite aplicar una función a toda la secuencia de forma paralela.

In [10]:
parallel_result = dview.map_sync(cube, range(1,7))
print parallel_result

[1, 8, 27, 64, 125, 216]


In [11]:
serial_result == parallel_result

True

3.- Determinar que partes de un programa que encuentra los primeros 5000 primos de Mersenne son paralelizables y sugerir como podrian paralelizarlo

In [25]:
PrimeQ=lambda x: all(x % i != 0 for i in range(int(x**0.5)+1)[2:]) and x>1

> Se puede paralelizar la parte del for, ya que éste consume demaciados recursos (memoria en este caso) al iterar y hacer calculos para cada elemento de la lista, secuencialmente. Se pudiera paralelizar esta parte dividiendo la lista en listas más pequeñas, aplicando la función a cada una de ellas, de froma paralela.

4.- Aplicar el metodo apply y/o map descritos aqui al programa que selecciona los primos de Mersenne de entre los primeros 5000 numeros enteros

In [12]:
def PrimeQ2(x):
    primos = []
    import numpy as np
    lista = np.arange(np.int(x**0.5)+1)[2:]
    residuos = np.mod(x, lista)
    if (np.all(residuos)):
        return x
    else:
        return 0

In [13]:
lista = dview.map_sync(PrimeQ2, 2**np.arange(10)-1)

In [14]:
print lista

[0, 1, 3, 7, 0, 31, 0, 127, 0, 0]
